In [13]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import cv2
import numpy as np
from PIL import Image

In [14]:
model = torchvision.models.squeezenet1_1(pretrained=True)
model.eval()

/Users/jakehenderson/nocuments/code/projects/ml-daily/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/jakehenderson/nocuments/code/projects/ml-daily/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): MaxPool2d

In [17]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [29]:
image = Image.open("mskasey.png")
input_tensor = transform(image).unsqueeze(0)
print(input_tensor.shape)
image = image.convert("RGB")
input_tensor = transform(image).unsqueeze(0)

torch.Size([1, 4, 224, 224])


In [30]:
def get_feature_map_hook(module, input, output):
    global feature_map
    feature_map = output

In [31]:
model.features[12].expand1x1.register_forward_hook(get_feature_map_hook)

In [32]:
with torch.no_grad():
    output = model(input_tensor)

In [33]:
# get class activation map
activation_map = feature_map.squeeze().mean(0).numpy()
activation_map = cv2.resize(activation_map, (image.width, image.height))
# normalize
activation_map = (activation_map - activation_map.min()) / (activation_map.max() - activation_map.min())

In [34]:
# Threshold & get bounding box
_, heatmap = cv2.threshold(activation_map, 0.6, 1, cv2.THRESH_BINARY)
contours, _ = cv2.findContours((heatmap * 255).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [35]:
# Draw bounding box
image = np.array(image)
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)

In [36]:
cv2.imwrite("squeezenet_bbox.jpg", image)
print("Bounding box saved as squeezenet_bbox.jpg")

Bounding box saved as squeezenet_bbox.jpg


=========================================================================================================================================================================================================================================================================================================================================================

In [ ]:
dummy_input = torch.randn(1, 3, 224, 224)
onnx_path = "squeezenet.onnx"
torch.onnx.export(model, dummy_input, onnx_path, verbose=True)
print(f"squeezenet saved as {onnx_path}")

check model size and ram usage

In [ ]:
import os
import onnxruntime 
import psutil 

In [ ]:
file_size = os.path.getsize(onnx_path) / (1024 * 1024)
print(f"ONNX model size: {file_size:.2f} MB")


In [ ]:
#check ram of onnx runtime
sessions = onnxruntime.InferenceSession(onnx_path)
process = psutil.Process()
memory_usage = process.memory_info().rss / (1024 * 1024)
print(f"Memory usage: {memory_usage:.2f} MB")

In [ ]:
# quantize
from onnxruntime.quantization import quantize_dynamic, QuantType

quantized_model_path = "squeezenet_quant.onnx"
quantize_dynamic(onnx_path, quantized_model_path, weight_type=QuantType.QUInt8)
file_size_q = os.path.getsize(quantized_model_path) / (1024 * 1024)
print(f"Quantized ONNX model size: {file_size_q:.2f} MB")

In [37]:
import onnxruntime

# Load ONNX model
session = onnxruntime.InferenceSession("squeezenet_quant.onnx")

# Convert image to ONNX format
input_data = input_tensor.numpy()
input_name = session.get_inputs()[0].name

# Run inference
outputs = session.run(None, {input_name: input_data})

# Get class activation map (similar to previous method)
activation_map = feature_map.squeeze().mean(0).numpy()
activation_map = cv2.resize(activation_map, (image.shape[1], image.shape[0]))
activation_map = (activation_map - activation_map.min()) / (activation_map.max() - activation_map.min())

# Get bounding box
_, heatmap = cv2.threshold(activation_map, 0.6, 1, cv2.THRESH_BINARY)
contours, _ = cv2.findContours((heatmap * 255).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green for quantized

cv2.imwrite("quantized_squeezenet_bbox.jpg", image)
print("Bounding box saved as quantized_squeezenet_bbox.jpg")


Bounding box saved as quantized_squeezenet_bbox.jpg
